Importing Datasets

In [2]:
from os import read
import pandas as pd 
df_nhanes = pd.read_csv("/Users/arjunanand/Documents/SE_Project/ClinicConnect/AI-Model/data/Nhanes_2013_2014.csv")
df_convo = pd.read_csv("/Users/arjunanand/Documents/SE_Project/ClinicConnect/AI-Model/data/Doctor-HealthCare-100k.csv")

Cleaning Datasets

In [2]:
from pdfminer.high_level import extract_text
import re

text = extract_text("/Users/arjunanand/Documents/SE_Project/ClinicConnect/AI-Model/data/oasis.pdf")

# Remove common patterns
patterns = [
    r"PRA Disclosure Statement.*?Baltimore, Maryland 21244-1850\.",
    r"Page \d+ of \d+",
    r"\*{5}CMS Disclaimer\*{5}.*?retained\.",  # Disclaimer block
    r"OASIS-E1 All Items Effective.*?Centers for Medicare & Medicaid Services",
    r"\n\s*\n"  # Empty lines
]

for pattern in patterns:
    text = re.sub(pattern, "", text, flags=re.DOTALL)

# Save cleaned text
with open("clean_oasis.txt", "w") as f:
    f.write(text)

In [3]:
import pandas as pd

df = pd.read_csv("/Users/arjunanand/Documents/SE_Project/ClinicConnect/AI-Model/data/Doctor-HealthCare-100k.csv")

#Remove duplicates
df = df.drop_duplicates(subset=["input"])

#Trim whitespace
df["output"] = df["output"].str.strip()

#Filter invalid responses
df = df[df["output"].str.contains("Chat Doctor|ENT evaluation|I hope", na=False)]

#Add safety disclaimers
df["output"] = df["output"] + "\n\nDISCLAIMER: This is simulated advice. Consult a real doctor."